In [0]:
#First try with the example of the research paper 

from pyspark import SparkContext

graph = sc.parallelize([(0, 1), (1, 2), (1, 3), (3, 4), (5, 6), (6, 7)])
counter_new_pair = sc.accumulator(1)

def reducer(node):
  minimum = node[0]
  value_List = []
  joined = []
  
  for value in node[1]:
    if value<minimum:
      minimum = value
    value_List.append(value)
  if minimum<node[0]:
    joined.append((node[0],minimum))
    for value in value_List:
      if minimum != value:
        counter_new_pair.add(1)
        joined.append((value, minimum))
  return joined

while counter_new_pair.value > 0:
  counter_new_pair = sc.accumulator(0)

  #CCF-Iterate
  mapping_1 = graph.map(lambda node : (node[0], node[1]))
  mapping_2 = graph.map(lambda node : (node[1], node[0]))
  fusion = mapping_1.union(mapping_2)
  fusion = fusion.groupByKey().map(lambda node : (node[0], list(node[1])))
  joined = fusion.flatMap(lambda node: reducer(node))
  
  # CCF-Dedup
  graph = joined.map(lambda node : ((node[0], node[1]), None)).groupByKey()
  graph = graph.map(lambda x: (x[0][0], x[0][1]))
  graph.collect()

In [0]:
import time

time1 = time.time()

lines = sc.textFile("/FileStore/tables/web_Google-12.txt")
# to convert the input line in two integers corresponding to the nodes of this edge
rows = lines.map(lambda x : x.strip().split()) 
graph = rows.map(lambda x : (int(x[0]), int(x[1])))

counter_new_pair = sc.accumulator(1)

while counter_new_pair.value > 0:
  counter_new_pair = sc.accumulator(0)

  #CCF-Iterate
  mapping_1 = graph.map(lambda node : (node[0], node[1]))
  mapping_2 = graph.map(lambda node : (node[1], node[0]))
  fusion = mapping_1.union(mapping_2)
  fusion = fusion.groupByKey().map(lambda node : (node[0], list(node[1])))
  joined = fusion.flatMap(lambda node: reducer(node))
  
  # CCF-Dedup
  graph = joined.map(lambda node : ((node[0], node[1]), None)).groupByKey()
  graph = graph.map(lambda x: (x[0][0], x[0][1]))
  graph.collect()
  
time2 = time.time()

print("It took {:.2f} second".format(time2-time1))

It took 783.77 second